In [0]:
%sql
--Counting rows by Gold Table(sanity check)

SELECT 'FactOrders'     AS table_name, COUNT(*) AS row_count FROM databricks_cat.gold.FactOrders
UNION ALL
SELECT 'DimCustomers'   AS table_name, COUNT(*) AS row_count FROM databricks_cat.gold.DimCustomers
UNION ALL
SELECT 'DimProducts'    AS table_name, COUNT(*) AS row_count FROM databricks_cat.gold.DimProducts;


In [0]:
%sql
--Orders don't have DimcustomerKey or DimProductKey
SELECT
  SUM(CASE WHEN DimCustomerKey IS NULL THEN 1 ELSE 0 END) AS orders_without_customer,
  SUM(CASE WHEN DimProductKey  IS NULL THEN 1 ELSE 0 END) AS orders_without_product,
  COUNT(*) AS total_orders
FROM databricks_cat.gold.FactOrders;

In [0]:
%sql
--Daily sales

SELECT
  order_date,
  ROUND(SUM(total_amount), 2)              AS total_sales,
  ROUND(SUM(quantity), 2)                  AS total_units,
  COUNT(DISTINCT order_id)       AS num_orders
FROM databricks_cat.gold.FactOrders
GROUP BY order_date
ORDER BY order_date;


In [0]:
%sql
--Daily sales by month

SELECT
  date_trunc('month', order_date) AS year_month,
  ROUND(SUM(total_amount),2)               AS total_sales,
  SUM(quantity)                   AS total_units,
  COUNT(DISTINCT order_id)        AS num_orders
FROM databricks_cat.gold.FactOrders
GROUP BY date_trunc('month', order_date)
ORDER BY year_month;


In [0]:
%sql
-- Top ten clients per sales

SELECT
  c.customer_id,
  c.full_name,          
  c.domain,             
  ROUND(SUM(f.total_amount),2)        AS total_sales,
  COUNT(DISTINCT f.order_id) AS num_orders,
  SUM(f.quantity)            AS total_units
FROM databricks_cat.gold.FactOrders   AS f
JOIN databricks_cat.gold.DimCustomers AS c
  ON f.DimCustomerKey = c.DimCustomerKey  
GROUP BY
  c.customer_id,
  c.full_name,
  c.domain
ORDER BY total_sales DESC
LIMIT 10;


In [0]:
%sql
-- Sales per email domain

SELECT
  c.domain,
  COUNT(DISTINCT c.customer_id) AS num_customers,
  COUNT(DISTINCT f.order_id)    AS num_orders,
  ROUND(SUM(f.total_amount),2)           AS total_sales,
  SUM(f.quantity)               AS total_units
FROM databricks_cat.gold.FactOrders   AS f
JOIN databricks_cat.gold.DimCustomers AS c
  ON f.DimCustomerKey = c.DimCustomerKey
GROUP BY c.domain
ORDER BY total_sales DESC;


In [0]:
%sql
-- Sales per category and brand

SELECT
  p.category,
  p.brand,
  ROUND(SUM(f.total_amount),2)           AS total_sales,
  SUM(f.quantity)               AS total_units,
  COUNT(DISTINCT f.order_id)    AS num_orders
FROM databricks_cat.gold.FactOrders   AS f
JOIN databricks_cat.gold.DimProducts  AS p
  ON f.DimProductKey = p.product_id
GROUP BY
  p.category,
  p.brand
ORDER BY total_sales DESC;


In [0]:
%sql
--Top 10 products by sales

SELECT
  p.product_id,
  p.product_name,
  p.category,
  p.brand,
  ROUND(SUM(f.total_amount),2)           AS total_sales,
  SUM(f.quantity)               AS total_units,
  COUNT(DISTINCT f.order_id)    AS num_orders
FROM databricks_cat.gold.FactOrders  AS f
JOIN databricks_cat.gold.DimProducts AS p
  ON f.DimProductKey = p.product_id  
GROUP BY
  p.product_id,
  p.product_name,
  p.category,
  p.brand
ORDER BY total_sales DESC
LIMIT 10;


In [0]:
%sql
--Current income vs. potential income with discounted price

SELECT
  date_trunc('month', f.order_date) AS year_month,
  ROUND(SUM(f.total_amount),2)                         AS sales_at_list_price,
  ROUND(SUM(f.quantity * p.discount_price),2)          AS sales_at_discount_price,
  ROUND(SUM(f.total_amount) -
    SUM(f.quantity * p.discount_price),2)        AS diff_if_discount
FROM databricks_cat.gold.FactOrders  AS f
JOIN databricks_cat.gold.DimProducts AS p
  ON f.DimProductKey = p.product_id  
GROUP BY date_trunc('month', f.order_date)
ORDER BY year_month;
